# PUZZLE ONE

In [1]:
import re
import pandas as pd
custs = pd.read_csv('noahs-customers.csv')

In [2]:
custs.head()

,customerid,name,address,citystatezip,birthdate,phone
0,1001,Jack Quinn,201 E Park St,"Los Angeles, CA 91343",1960-05-14,805-287-8515
1,1002,David Powell,224C Tysens Ln,"Staten Island, NY 10306",1978-04-04,516-768-1652
2,1003,Carrie Green,1608 W 53rd Way,"Tampa, FL 33614",1969-01-21,727-209-0470
3,1004,Steven Miller,178½ E 7th St,"Manhattan, NY 10009",1953-08-17,607-941-9563
4,1005,Christine Powers,270 W 242nd St,"Bronx, NY 10463",1983-06-06,212-759-9043


In [3]:
LETTERS_DIGITS = {'ABC':'2', 'DEF':'3', 'GHI':'4', 'JKL':'5',
                'MNO':'6', 'PQRS':'7', 'TUV':'8', 'WXYZ':'9'}

In [4]:
for name0, phone0 in zip(custs['name'], custs['phone']):
    name = re.findall(r' (\w+)$', name0.upper())
    if name:
        phone_match = list(map(lambda x: [LETTERS_DIGITS[k] for k in LETTERS_DIGITS.keys() if x in k][0], name[0])) 
        phone = re.findall(r'\d', phone0)
        if phone == phone_match:
            print(phone0)
            break

488-836-2374


# PUZZLE TWO

In [5]:
products = pd.read_csv('noahs-products.csv')

In [6]:
products.head()

,sku,desc,wholesale_cost
0,DLI0002,Smoked Whitefish Sandwich,9.33
1,PET0005,"Vegan Cat Food, Turkey & Chicken",4.35
2,HOM0018,Power Radio (red),21.81
3,KIT0034,Azure Ladle,2.81
4,PET0041,"Gluten-free Cat Food, Pumpkin & Pumpkin",4.60


In [12]:
orders = pd.read_csv('noahs-orders.csv')

In [13]:
orders.head()

,orderid,customerid,ordered,shipped,items,total
0,1001,4308,2017-01-31 00:32:19,2017-01-31 07:15:00,NaN,25.52
1,1002,11683,2017-01-31 00:58:31,2017-01-31 18:00:00,NaN,35.33
2,1003,5676,2017-01-31 01:34:40,2017-01-31 09:00:00,NaN,30.79
3,1004,3097,2017-01-31 02:31:24,2017-01-31 19:45:00,NaN,77.60
4,1005,10374,2017-01-31 02:46:09,2017-01-31 14:45:00,NaN,109.04


In [35]:
orders['shipped']

0         2017-01-31 07:15:00
1         2017-01-31 18:00:00
2         2017-01-31 09:00:00
3         2017-01-31 19:45:00
4         2017-01-31 14:45:00
                 ...         
214202    2022-12-17 16:45:00
214203    2022-12-16 21:25:51
214204    2022-12-18 18:00:00
214205    2022-12-18 08:30:00
214206    2022-12-18 15:30:00
Name: shipped, Length: 214207, dtype: object

In [14]:
order_items = pd.read_csv('noahs-orders_items.csv')

In [15]:
order_items.head()

,orderid,sku,qty,unit_price
0,1001,COL0820,1,25.52
1,1002,TOY8907,1,12.92
2,1002,KIT5813,1,7.99
3,1002,KIT3981,2,7.21
4,1003,KIT7098,1,12.53


In [20]:
big_t = products.merge(order_items, how='inner', on='sku').merge(orders, how='inner', on='orderid',)\
        .merge(custs, how='inner', on='customerid') 

In [42]:
idxs = []
for phone, prod, name, shipped in zip(big_t['phone'], big_t['desc'], big_t['name'], big_t['shipped']):
    if 'rug' in prod.lower() and ''.join(re.findall(r'([A-Z])[a-z]+', name)) == 'JD' and shipped < '2017-06-01':
        print(phone, name)

516-994-3023 Jennifer Daniels
212-771-8924 Jeremy Davis
212-771-8924 Jeremy Davis
212-771-8924 Jeremy Davis
212-771-8924 Jeremy Davis
212-771-8924 Jeremy Davis


# PUZZLE THREE

In [74]:
custs[custs['name'] == 'Jeremy Davis']
    

,customerid,name,address,citystatezip,birthdate,phone,birthyear,birthday
3163,4164,Jeremy Davis,134-10 Foch Blvd,"South Ozone Park, NY 11420",1995-07-26,212-771-8924,1995,07-26


In [73]:
custs['birthyear'] = custs['birthdate'].apply(lambda x: int(x[:4]))
custs['birthday'] =  custs['birthdate'].apply(lambda x: x[5:])

In [83]:
dog_years = {1922 + 12 * k for k in range(12)}
dog_years = list(map(str, dog_years))
candidates = custs[((custs['birthyear'] - 1982) % 12 == 0) & ('03-20' <= custs['birthday']) & (custs['birthday'] <= '04-20')]
len(candidates)


87

In [88]:
candidates['hood'] = candidates['citystatezip'].apply(lambda x: re.findall(r'^(.+),', x)[0])

C:\Users\etien\AppData\Local\Temp/ipykernel_26500/2512976097.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  candidates['hood'] = candidates['citystatezip'].apply(lambda x: re.findall(r'^(.+),', x)[0])


In [90]:
candidates[candidates['hood'] == 'South Ozone Park']

,customerid,name,address,citystatezip,birthdate,phone,birthyear,birthday,hood
1273,2274,Brent Nguyen,109-19 110th St,"South Ozone Park, NY 11420",1958-03-25,516-636-7397,1958,03-25,South Ozone Park


# PUZZLE FOUR

In [91]:
big_t[big_t['customerid'] == 2274]

,sku,desc,wholesale_cost,orderid,qty,unit_price,customerid,ordered,shipped,items,total,name,address,citystatezip,birthdate,phone
426464,COL4384,Noah's Poster (white),7.52,123932,1,10.62,2274,2020-06-12 21:19:07,2020-06-14 16:15:00,NaN,24.80,Brent Nguyen,109-19 110th St,"South Ozone Park, NY 11420",1958-03-25,516-636-7397
426465,KIT4742,Manual Coffee Bowl,11.27,123932,1,14.18,2274,2020-06-12 21:19:07,2020-06-14 16:15:00,NaN,24.80,Brent Nguyen,109-19 110th St,"South Ozone Park, NY 11420",1958-03-25,516-636-7397
426466,HOM8601,Rug Cleaner,3.51,20129,2,4.12,2274,2017-08-12 16:18:23,2017-08-12 16:18:23,NaN,8.24,Brent Nguyen,109-19 110th St,"South Ozone Park, NY 11420",1958-03-25,516-636-7397


In [104]:
big_t['pastries?'] = big_t['sku'].apply(lambda x: 'BKY' in x)

In [116]:
big_t['early_shopper'] = big_t['shipped'].apply(lambda x: '04:30:00' < x[-8:] < '05:00:00')

In [117]:
big_t[big_t['early_shopper'] & big_t['pastries?']]

,sku,desc,wholesale_cost,orderid,qty,unit_price,customerid,ordered,shipped,items,total,name,address,citystatezip,birthdate,phone,pastries?,early_shopper
108775,BKY4022,Sesame Bialy,5.86,11996,2,7.40,4640,2017-05-22 00:50:31,2017-05-22 04:45:00,NaN,36.18,Felicia Davis,208½ W 94th St,"Manhattan, NY 10025",1950-05-20,332-718-0838,True,True
127649,BKY0403,Poppyseed Linzer Cookie,4.87,78689,1,5.53,9818,2019-03-18 01:29:09,2019-03-18 04:45:00,NaN,74.21,Glenn Alvarez,110A W 136th St,"Manhattan, NY 10030",1964-03-15,585-729-2248,True,True
144372,BKY5887,Sesame Bagel,6.38,26173,1,7.89,5105,2017-10-11 04:58:38,2017-10-11 04:58:38,NaN,389.65,Heather Brown,2305 Arthur Ave,"Bronx, NY 10458",1970-02-28,929-518-7269,True,True
309844,BKY4940,Raspberry Hamentash,5.31,52609,1,6.76,8078,2018-07-02 01:29:09,2018-07-02 04:45:00,NaN,6.76,Nick Brown,258A W 230th St,"Bronx, NY 10463",1964-02-22,838-347-2025,True,True
367348,BKY2239,Poppyseed Mandelbrot,5.11,96375,1,6.11,2063,2019-09-11 00:56:49,2019-09-11 04:45:00,NaN,6.11,Katie Wilkins,394 W 33rd St,"Manhattan, NY 10001",1968-04-06,838-216-1201,True,True
375720,BKY0455,Raspberry Rugelach,5.24,174534,1,6.36,5375,2021-11-02 04:41:47,2021-11-02 04:41:47,NaN,6.36,Christina Booker,1127 Grinnell Pl,"Bronx, NY 10474",1981-01-08,718-649-9036,True,True
375814,BKY3104,Caraway Puff,4.36,201780,1,5.39,5375,2022-08-03 04:40:13,2022-08-03 04:40:13,NaN,5.39,Christina Booker,1127 Grinnell Pl,"Bronx, NY 10474",1981-01-08,718-649-9036,True,True
375823,BKY4022,Sesame Bialy,5.86,196511,1,7.72,5375,2022-06-11 04:44:42,2022-06-11 04:44:42,NaN,7.72,Christina Booker,1127 Grinnell Pl,"Bronx, NY 10474",1981-01-08,718-649-9036,True,True
375866,BKY7160,Caraway Twist,6.15,199892,2,7.25,5375,2022-07-15 04:38:38,2022-07-15 04:38:38,NaN,14.50,Christina Booker,1127 Grinnell Pl,"Bronx, NY 10474",1981-01-08,718-649-9036,True,True
